In [302]:
import os

import numpy
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from functools import partial
from sklearn.metrics import mean_squared_error, f1_score

In [301]:
DATA_DIR = "/home/gangda/workspace/ds-richter/data"

X = pd.read_csv(DATA_DIR + '/train_values.csv', index_col='building_id')
X_test = pd.read_csv(DATA_DIR + '/test_values.csv', index_col='building_id')
y = pd.read_csv(DATA_DIR + '/train_labels.csv', index_col='building_id')

def mean_prob(prob):
    df = pd.DataFrame()
    df_prob = pd.DataFrame(prob)
    if prob.shape[0] == X.shape[0] :
        df_prob = df_prob.join(X['geo_level_3_id'].reset_index(drop=True))
    elif prob.shape[0] == X_test.shape[0]:
        df_prob = df_prob.join(X_test['geo_level_3_id'].reset_index(drop=True))
    else:
        assert False, 'prob.shape[0] should be align with X or X_test'
    df[0] = df_prob.groupby('geo_level_3_id')[0].transform('mean')
    df[1] = df_prob.groupby('geo_level_3_id')[1].transform('mean')
    df[2] = df_prob.groupby('geo_level_3_id')[2].transform('mean')
    return df.to_numpy()

In [285]:
DATA_DIR = "/home/gangda/workspace/ds-richter/data"

y_catboost_1 = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_train.npy'.format(3, 1))
y_catboost_2 = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_train.npy'.format(4, 2))
y_catboost_3 = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_train.npy'.format(5, 1))
y_catboost_3_2 = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_train.npy'.format(5, 2))
y_lightgbm_1 = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_train.npy'.format(3, 1))
y_lightgbm_2 = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_train.npy'.format(4, 1))
y_lightgbm_3 = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_train.npy'.format(5, 1))
y_xgboost_3 = np.load(DATA_DIR + '/intermediate/xgboost_{}fold_v{}_train.npy'.format(5, 1))


y_catboost_1_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_test.npy'.format(3, 1))
y_catboost_2_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_test.npy'.format(4, 2))
y_catboost_3_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_test.npy'.format(5, 1))
y_catboost_3_2_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_test.npy'.format(5, 2))
y_lightgbm_1_test = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_test.npy'.format(3, 1))
y_lightgbm_2_test = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_test.npy'.format(4, 1))
y_lightgbm_3_test = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_test.npy'.format(5, 1))
y_xgboost_3_test = np.load(DATA_DIR + '/intermediate/xgboost_{}fold_v{}_test.npy'.format(5, 1))

---

In [303]:
label = y.to_numpy().reshape(-1) - 1
y_onehot = np.eye(3)[label]

df = pd.DataFrame()
df_prob = pd.DataFrame(y_onehot)
df_prob = df_prob.join(X['geo_level_3_id'].reset_index(drop=True))
df[0] = df_prob.groupby('geo_level_3_id')[0].mean()
df[1] = df_prob.groupby('geo_level_3_id')[1].mean()
df[2] = df_prob.groupby('geo_level_3_id')[2].mean()

df.loc[df.sample(500).index, :] = 0
df = df.loc[X['geo_level_3_id'].to_numpy(), :]
# df.loc[df.sample(1000).index, :] = 0
gt_train = df.to_numpy()

df

,0,1,2
geo_level_3_id,,,
12198,0.000000,0.162162,0.837838
2812,0.062500,0.812500,0.125000
8973,0.029412,0.360294,0.610294
10694,0.032258,0.838710,0.129032
1488,0.008197,0.614754,0.377049
...,...,...,...
1621,0.000000,0.928571,0.071429
2060,0.000000,0.020408,0.979592
8163,0.045455,0.090909,0.863636


In [304]:
df = pd.DataFrame(y_onehot)
df = df.join(X['geo_level_3_id'].reset_index(drop=True))
mean_val = pd.DataFrame()
mean_val[0] = df.groupby('geo_level_3_id')[0].mean()
mean_val[1] = df.groupby('geo_level_3_id')[1].mean()
mean_val[2] = df.groupby('geo_level_3_id')[2].mean()

geo_id_train = X['geo_level_3_id'].unique()
geo_id_test = X_test['geo_level_3_id'].unique()
geo_id_diff = np.setdiff1d(geo_id_test, geo_id_train, assume_unique=True)
df_geo_diff = pd.DataFrame(index=geo_id_diff, data=np.zeros((geo_id_diff.shape[0], 3)))

df = pd.concat([mean_val, df_geo_diff])
df = df.loc[X_test['geo_level_3_id'].to_numpy(), :]
gt_test = df.to_numpy()

df

,0,1,2
11307,0.052632,0.263158,0.684211
11987,0.000000,1.000000,0.000000
10044,0.000000,0.000000,1.000000
633,0.571429,0.419355,0.009217
7970,0.000000,0.176471,0.823529
...,...,...,...
3623,0.194444,0.750000,0.055556
11907,0.000000,0.277778,0.722222
7712,0.090909,0.909091,0.000000
912,0.090909,0.681818,0.227273


In [305]:
# Count the number of all-zero rows
num_zero_rows = np.sum(np.all(gt_test == 0, axis=1))

# Print the result
print("Number of all-zero rows:", num_zero_rows)

Number of all-zero rows: 327


---

In [306]:
y_prob = np.stack([y_catboost_3, y_lightgbm_3])

y_prob = np.log1p(y_prob)
y_prob = np.sum(y_prob, axis=0)
y_pred = y_prob.argmax(axis=1) + 1
f1_score(y, y_pred, average='micro')

0.7537269619072835

In [290]:
y_prob = np.stack([y_catboost_3_test, y_lightgbm_3_test, y_xgboost_3_test])

y_prob = np.log1p(y_prob)
y_prob = np.sum(y_prob, axis=0)
y_pred = y_prob.argmax(axis=1) + 1
y_pred

array([3, 2, 2, ..., 2, 2, 1])

In [309]:
from sklearn.metrics import classification_report, confusion_matrix

# mixure
# print(classification_report(y, y_pred))
print(confusion_matrix(y, y_catboost_3.argmax(axis=1) + 1))
print(classification_report(y, y_catboost_3.argmax(axis=1) + 1))

print(confusion_matrix(y, y_lightgbm_3.argmax(axis=1) + 1))
print(classification_report(y, y_lightgbm_3.argmax(axis=1) + 1))

print(confusion_matrix(y, y_pred))
print(classification_report(y, y_pred))
# sns.distplot(y_pred)

[[ 13758  11036    330]
 [  5393 126094  16772]
 [   564  30526  56128]]
              precision    recall  f1-score   support

           1       0.70      0.55      0.61     25124
           2       0.75      0.85      0.80    148259
           3       0.77      0.64      0.70     87218

    accuracy                           0.75    260601
   macro avg       0.74      0.68      0.70    260601
weighted avg       0.75      0.75      0.75    260601

[[ 14299  10351    474]
 [  6075 124571  17613]
 [   680  29535  57003]]
              precision    recall  f1-score   support

           1       0.68      0.57      0.62     25124
           2       0.76      0.84      0.80    148259
           3       0.76      0.65      0.70     87218

    accuracy                           0.75    260601
   macro avg       0.73      0.69      0.71    260601
weighted avg       0.75      0.75      0.75    260601

[[ 14110  10651    363]
 [  5589 125756  16914]
 [   602  30060  56556]]
              preci

In [300]:
from sklearn.metrics import classification_report, confusion_matrix

# mixure
# print(classification_report(y, y_pred))
print(confusion_matrix(y, y_pred))
# sns.distplot(y_pred)

[[ 13660  11137    327]
 [  5095 126738  16426]
 [   502  30562  56154]]


In [259]:
class GeneralizedMeanBlender:
    """Combines multiple predictions using generalized mean"""
    def __init__(self, p_range=(-2,2)):
        """"""
        self.p_range = p_range
        self.p = None
        self.weights = None

    def _objective(self, trial, X, y):

        # create hyperparameters
        p = trial.suggest_float(f"p", *self.p_range)
        weights = [
            trial.suggest_float(f"w{i}", 0, 1)
            for i in range(X.shape[0] * 1)
        ]
        weights = np.reshape(weights, (X.shape[0], 1, 1))

        # blend predictions
        if p <= 0:
            blend_preds = np.log1p(X) * weights
        else:
            blend_preds = X**p * weights
        blend_preds = np.sum(blend_preds, axis=0)

        y_pred = blend_preds.argmax(axis=1) + 1
        return f1_score(y, y_pred, average='micro')

    def fit(self, X, y, n_trials=10):
        # optimize objective
        obj = partial(self._objective, X=X, y=y)
        study = optuna.create_study(direction="maximize")
        study.optimize(obj, n_trials=n_trials)
        # extract best weights
        if self.p is None:
            self.p = [v for k,v in study.best_params.items() if "p" in k][0]
        self.weights = np.array([v for k,v in study.best_params.items() if "w" in k])
        self.weights /= self.weights.sum()

        self.study = study

    def transform(self, X, return_prob=False):
        assert self.weights is not None and self.p is not None, \
            "Must call fit method before transform"
        weights = np.reshape(self.weights, (X.shape[0], 1, 1))
        if self.p <= 0:
            prob = np.log1p(X) * weights
        else:
            prob = X**self.p * weights
        prob = np.sum(prob, axis=0)

        if return_prob:
            return prob
        else:
            return prob.argmax(axis=1) + 1

    def fit_transform(self, X, y, **kwargs):
        self.fit(X, y, **kwargs)
        return self.transform(X)

In [292]:
X_prob = np.stack([
    y_catboost_3, y_lightgbm_3, y_xgboost_3
])
y_prob_test = np.stack([
    y_catboost_3_test, y_lightgbm_3_test, y_xgboost_3_test
])

gmb = GeneralizedMeanBlender()
gmb.fit(X_prob, y, n_trials=500)

[I 2023-04-21 19:17:18,113] A new study created in memory with name: no-name-879e478f-37de-403e-980b-14f43e8ae5ac
[I 2023-04-21 19:17:18,193] Trial 0 finished with value: 0.751627967659372 and parameters: {'p': -0.11467048551611914, 'w0': 0.8436779393519296, 'w1': 0.008422344985566266, 'w2': 0.856650329997316}. Best is trial 0 with value: 0.751627967659372.
[I 2023-04-21 19:17:18,263] Trial 1 finished with value: 0.7532511387139728 and parameters: {'p': -0.38724737490505223, 'w0': 0.15431049803686536, 'w1': 0.6878692314505306, 'w2': 0.7879581487956597}. Best is trial 1 with value: 0.7532511387139728.
[I 2023-04-21 19:17:18,346] Trial 2 finished with value: 0.7521882110966572 and parameters: {'p': 0.44274873212808474, 'w0': 0.6813897538311129, 'w1': 0.09790481934021822, 'w2': 0.7275130718431635}. Best is trial 1 with value: 0.7532511387139728.
[I 2023-04-21 19:17:18,429] Trial 3 finished with value: 0.7540684801669986 and parameters: {'p': 0.16869630659929458, 'w0': 0.4512235073676276, 

In [268]:
X_prob2 = np.stack([
    gmb.transform(X_prob, return_prob=True), gt_train
])
y_prob2_test = np.stack([
    gmb.transform(y_prob_test, return_prob=True), gt_test
])

gmb2 = GeneralizedMeanBlender()
gmb2.fit(X_prob2, y, n_trials=500)

[I 2023-04-19 23:24:14,753] A new study created in memory with name: no-name-8c9ec92c-c25e-403f-aef2-3276a56283ec
[I 2023-04-19 23:24:14,826] Trial 0 finished with value: 0.7543255781827392 and parameters: {'p': 1.3069103543662046, 'w0': 0.9363767685465416, 'w1': 0.9446372826314027}. Best is trial 0 with value: 0.7543255781827392.
[I 2023-04-19 23:24:14,890] Trial 1 finished with value: 0.7556954885054163 and parameters: {'p': -0.4851605348843875, 'w0': 0.9538433035135042, 'w1': 0.1575593770785897}. Best is trial 1 with value: 0.7556954885054163.
[I 2023-04-19 23:24:14,961] Trial 2 finished with value: 0.7565320163775273 and parameters: {'p': 0.24361611094684488, 'w0': 0.9032244329698128, 'w1': 0.7155604724796314}. Best is trial 2 with value: 0.7565320163775273.
[I 2023-04-19 23:24:15,033] Trial 3 finished with value: 0.7548474487818543 and parameters: {'p': 1.3484387472654653, 'w0': 0.9698426250502845, 'w1': 0.0686979621017475}. Best is trial 2 with value: 0.7565320163775273.
[I 2023-

In [297]:
f1_score(y, gmb.transform(X_prob), average='micro')

0.7547169811320754

In [278]:
print('gmb', gmb.weights, gmb.p)
print('gmb2', gmb2.weights, gmb2.p)

gmb [0.3371862  0.47654352 0.18627028] -0.39627827486783557
gmb2 [0.73591627 0.26408373] 0.12354957751640756


In [279]:
fig = optuna.visualization.plot_param_importances(gmb.study)
fig.show()

## Submit

In [296]:
test_preds = gmb.transform(y_prob_test)

submission_format = pd.read_csv(DATA_DIR + '/submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=test_preds,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission['damage_grade'] = my_submission['damage_grade'].astype(int)
my_submission.to_csv(DATA_DIR + '/submission/mixture_{}.csv'.format('cat_lgb_xgb_v2'))